# S3에 있는 파일럿 테스트->무료배포 시점 데이터로 교체

## 교체해야할 S3 객체 명, 타깃 데이터 이름 선별

In [3]:
import boto3
import os

s3 = boto3.client('s3')
bucket_name = 'schoolfriends-bym'
response = s3.list_objects_v2(Bucket=bucket_name)
for obj in response['Contents'] :
    print(obj)

{'Key': 'at_basic_inquiry_val/', 'LastModified': datetime.datetime(2025, 8, 29, 7, 59, 56, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'at_basic_inquiry_val/AT_BASIC_INQUIRY_VAL_202508291453.csv', 'LastModified': datetime.datetime(2025, 8, 29, 8, 2, 7, tzinfo=tzutc()), 'ETag': '"2d9719881bc5935b6cfeffc04e93c329"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 79095, 'StorageClass': 'STANDARD'}
{'Key': 'at_formula_code/', 'LastModified': datetime.datetime(2025, 8, 29, 8, 0, 26, tzinfo=tzutc()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'at_formula_code/AT_FORMULA_CODE_202508291455.csv', 'LastModified': datetime.datetime(2025, 8, 29, 8, 2, 26, tzinfo=tzutc()), 'ETag': '"7d0c7328055b7487783b240a191d2434"', 'Checks

In [12]:
# athena 데이터가 너무 많이 쌓여 있어 삭제

s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
prefix = "athena/"  # 지우고 싶은 경로

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

for obj in response['Contents']:
    print(obj['Key'])
    s3.delete_object(Bucket=bucket_name, Key=obj['Key'])

athena/e324b7be-dd06-4018-8376-ca928c52b723.csv.metadata
athena/e37eb326-fda0-4202-8b71-52b70eab5f64.csv
athena/e37eb326-fda0-4202-8b71-52b70eab5f64.csv.metadata
athena/e39a4920-724c-4180-bfb0-0732427cd266.csv
athena/e39a4920-724c-4180-bfb0-0732427cd266.csv.metadata
athena/e39b2bd6-0fa5-4de1-92a8-d7763ae5a276.csv
athena/e39b2bd6-0fa5-4de1-92a8-d7763ae5a276.csv.metadata
athena/e39c3082-bef3-4883-be47-9080f43be1b0.csv
athena/e39c3082-bef3-4883-be47-9080f43be1b0.csv.metadata
athena/e3a2ce0d-da8e-4696-9823-63fcb7ace034.csv
athena/e3a2ce0d-da8e-4696-9823-63fcb7ace034.csv.metadata
athena/e3b285d1-3ac7-42ed-ac7c-2109722d08a3.csv
athena/e3b285d1-3ac7-42ed-ac7c-2109722d08a3.csv.metadata
athena/e3c57f78-db46-4f98-838a-549ddbe21147.csv
athena/e3c57f78-db46-4f98-838a-549ddbe21147.csv.metadata
athena/e3d7676d-2383-42b0-a6ad-311a42044d84.csv
athena/e3d7676d-2383-42b0-a6ad-311a42044d84.csv.metadata
athena/e3e754ce-23c4-4d80-9736-c0783403b16c.csv
athena/e3e754ce-23c4-4d80-9736-c0783403b16c.csv.metadat

In [40]:
# 프리픽스 확인
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
response = s3.list_objects_v2(Bucket=bucket_name)
target_prefix = []
for obj in response['Contents']:
    # 마지막에 '/'가 있는지 확인
    if obj['Key'].endswith('/'):
        target_prefix.append(obj['Key'])
    else:
        pass
print(target_prefix)

['at_basic_inquiry_val/', 'at_formula_code/', 'at_low_scale/', 'at_question/', 'at_question_choice/', 'at_question_extra/', 'at_scale_code/', 'at_sub_psy_item/', 'at_user_formula_score/', 'at_user_scale_score/', 'at_user_testing_paper/', 'at_user_testing_paper_pn/', 'at_user_testing_paper_sct/', 'at_user_testing_paper_tr/', 'jar/', 'member/', 'psy_ord_class/', 'result/aggression_norms/', 'school_class/']


In [43]:
# 교체할 csv 파일 목록
local_directory = "C:/Users/USER/aws_etl/schoolfriends/250930_extraction_data/"
file_list = os.listdir(local_directory)
target_csv = []
for file in file_list:
    if file.endswith('.csv'):
        target_csv.append(file)
print(target_csv)

# 끝에 있는 _와 숫자 제거
cleaned_target_csv = []
for cleaned_file in target_csv:
    if '_' in cleaned_file:
        cleaned_file = cleaned_file.rsplit('_', 1)[0] # 마지막 '_' 기준으로 분리하여 앞부분만 취함
    cleaned_target_csv.append(cleaned_file)
print(cleaned_target_csv)

['AT_USER_TESTING_PAPER_202509301206.csv', 'AT_USER_TESTING_PAPER_PN_202509300911.csv', 'AT_USER_TESTING_PAPER_SCT_202509301206.csv', 'AT_USER_TESTING_PAPER_TR_202509301206.csv', 'MEMBER_202509301207.csv', 'PSY_CLASS_202509300905.csv', 'PSY_ESTIMATE_202509301207.csv', 'PSY_ORD_202509301207.csv', 'PSY_ORD_CLASS_202509301207.csv', 'PSY_TARGET_DETAIL_202509300907.csv', 'SCHOOL_CLASS_202509300905.csv', 'SCHOOL_INFO_202509300903.csv', 'SCHOOL_STUDENT_202509301207.csv', 'SCHOOL_TEACHER_202509301207.csv']
['AT_USER_TESTING_PAPER', 'AT_USER_TESTING_PAPER_PN', 'AT_USER_TESTING_PAPER_SCT', 'AT_USER_TESTING_PAPER_TR', 'MEMBER', 'PSY_CLASS', 'PSY_ESTIMATE', 'PSY_ORD', 'PSY_ORD_CLASS', 'PSY_TARGET_DETAIL', 'SCHOOL_CLASS', 'SCHOOL_INFO', 'SCHOOL_STUDENT', 'SCHOOL_TEACHER']


In [24]:
# target_csv 이름 소문자로 변환
cleaned_target_csv = [file.lower() for file in cleaned_target_csv]
print(cleaned_target_csv)

['at_user_testing_paper', 'at_user_testing_paper_pn', 'at_user_testing_paper_sct', 'at_user_testing_paper_tr', 'member_', 'psy_class', 'psy_estimate', 'psy_ord', 'psy_ord_class', 'psy_target_detail', 'school_class', 'school_info', 'school_student', 'school_teacher']


In [42]:
# target_csv 이름에 해당하는 프리픽스 찾아가서 업로드 되도록
target_upload_dict = {}
for key, value in zip(cleaned_target_csv, target_csv):
    if key in value.lower():
        target_upload_dict[key] = value
    else:
        print(f"No match for {value} in prefixes.")
print(target_upload_dict)

# 파일 업로드
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
for prefix in target_prefix:
    for key, value in target_upload_dict.items():
        if key in prefix:
            file_path = os.path.join(prefix, value)
            print(file_path)
            try:
                s3.upload_file(value, bucket_name, file_path)
                print(f"Uploaded {value} to {file_path}")
            except Exception as e:
                print(f"Error uploading {value} to {file_path}: {e}")

{'at_user_testing_paper': 'AT_USER_TESTING_PAPER_202509301206.csv', 'at_user_testing_paper_pn': 'AT_USER_TESTING_PAPER_PN_202509300911.csv', 'at_user_testing_paper_sct': 'AT_USER_TESTING_PAPER_SCT_202509301206.csv', 'at_user_testing_paper_tr': 'AT_USER_TESTING_PAPER_TR_202509301206.csv', 'member_': 'MEMBER__202509301207.csv', 'psy_class': 'PSY_CLASS_202509300905.csv', 'psy_estimate': 'PSY_ESTIMATE_202509301207.csv', 'psy_ord': 'PSY_ORD_202509301207.csv', 'psy_ord_class': 'PSY_ORD_CLASS_202509301207.csv', 'psy_target_detail': 'PSY_TARGET_DETAIL_202509300907.csv', 'school_class': 'SCHOOL_CLASS_202509300905.csv', 'school_info': 'SCHOOL_INFO_202509300903.csv', 'school_student': 'SCHOOL_STUDENT_202509301207.csv', 'school_teacher': 'SCHOOL_TEACHER_202509301207.csv'}
at_user_testing_paper/AT_USER_TESTING_PAPER_202509301206.csv
Uploaded AT_USER_TESTING_PAPER_202509301206.csv to at_user_testing_paper/AT_USER_TESTING_PAPER_202509301206.csv
at_user_testing_paper_pn/AT_USER_TESTING_PAPER_202509301

## in 연산자 판단으로 뭐때문인지 몰라도 잘못 들어가거나 안들어감

In [46]:
#! 폴더 생성을 안거치고 바로 업로드 된 경우 end(/)로 판단이 안되어 target_prefix에 안들어가서 업로드가 안됨
#! in 연산자 판단으로 비슷한 이름이 있는 경우 경로가 잘못 설정되어 들어감

# target_prefix의 과거 데이터 삭제

In [51]:
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
response = s3.list_objects_v2(Bucket=bucket_name)
target_prefix_set = set()
for obj in response['Contents']:
    # '/' 포함되어 있는 키값
    if '/' in obj['Key']:
        target_prefix_set.add(obj['Key'].split('/')[0] + '/')
    else:
        print(f"Not a prefix: {obj['Key']}")

In [52]:
target_prefix_set

{'at_basic_inquiry_val/',
 'at_formula_code/',
 'at_low_scale/',
 'at_question/',
 'at_question_choice/',
 'at_question_extra/',
 'at_scale_code/',
 'at_sub_psy_item/',
 'at_user_formula_score/',
 'at_user_scale_score/',
 'at_user_testing_paper/',
 'at_user_testing_paper_pn/',
 'at_user_testing_paper_sct/',
 'at_user_testing_paper_tr/',
 'jar/',
 'member/',
 'psy_class/',
 'psy_estimate/',
 'psy_ord/',
 'psy_ord_class/',
 'psy_target/',
 'result/',
 'school_class/',
 'school_info/',
 'school_student/',
 'school_teacher/'}

In [64]:
# 업로드할 파일 목록
target_file_names = []
local_directory = "C:/Users/USER/aws_etl/schoolfriends/250930_extraction_data/"
file_list = os.listdir(local_directory)
for file_name in file_list:
    if file_name.endswith(".csv"):
        target_file_names.append(file_name)
print(target_file_names)

['AT_USER_TESTING_PAPER_202509301206.csv', 'AT_USER_TESTING_PAPER_PN_202509300911.csv', 'AT_USER_TESTING_PAPER_SCT_202509301206.csv', 'AT_USER_TESTING_PAPER_TR_202509301206.csv', 'MEMBER_202509301207.csv', 'PSY_CLASS_202509300905.csv', 'PSY_ESTIMATE_202509301207.csv', 'PSY_ORD_202509301207.csv', 'PSY_ORD_CLASS_202509301207.csv', 'PSY_TARGET_202509300907.csv', 'SCHOOL_CLASS_202509300905.csv', 'SCHOOL_INFO_202509300903.csv', 'SCHOOL_STUDENT_202509301207.csv', 'SCHOOL_TEACHER_202509301207.csv']


In [69]:
# 최신 데이터가 업로드된 프리픽스 확인
import re

cleaned_target_prefix = []
for prefix in target_prefix_set:
    for file_name in target_file_names:
        print(prefix.lower().rstrip('/'))
        print(re.sub(r'_?\d+', '', file_name.lower()).rstrip('.csv'))
        if prefix.lower().rstrip('/') == re.sub(r'_?\d+', '', re.sub(r'\.csv$', '', file_name.lower())):
            cleaned_target_prefix.append(prefix)
        else:
            print(f"No match for {file_name} in prefixes.")
print(cleaned_target_prefix)

member
at_user_testing_paper
No match for AT_USER_TESTING_PAPER_202509301206.csv in prefixes.
member
at_user_testing_paper_pn
No match for AT_USER_TESTING_PAPER_PN_202509300911.csv in prefixes.
member
at_user_testing_paper_sct
No match for AT_USER_TESTING_PAPER_SCT_202509301206.csv in prefixes.
member
at_user_testing_paper_tr
No match for AT_USER_TESTING_PAPER_TR_202509301206.csv in prefixes.
member
member
member
psy_cla
No match for PSY_CLASS_202509300905.csv in prefixes.
member
psy_estimate
No match for PSY_ESTIMATE_202509301207.csv in prefixes.
member
psy_ord
No match for PSY_ORD_202509301207.csv in prefixes.
member
psy_ord_cla
No match for PSY_ORD_CLASS_202509301207.csv in prefixes.
member
psy_target
No match for PSY_TARGET_202509300907.csv in prefixes.
member
school_cla
No match for SCHOOL_CLASS_202509300905.csv in prefixes.
member
school_info
No match for SCHOOL_INFO_202509300903.csv in prefixes.
member
school_student
No match for SCHOOL_STUDENT_202509301207.csv in prefixes.
memb

In [73]:
cleaned_target_prefix

['at_user_testing_paper/',
 'at_user_testing_paper_pn/',
 'at_user_testing_paper_sct/',
 'at_user_testing_paper_tr/',
 'member/',
 'psy_class/',
 'psy_estimate/',
 'psy_ord/',
 'psy_ord_class/',
 'psy_target/',
 'school_class/',
 'school_info/',
 'school_student/',
 'school_teacher/']

In [102]:
# 타겟 프리픽스의 과거 데이터 목록 조회
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"

for prefix in cleaned_target_prefix:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    print()
    print(f"Objects in prefix: {prefix}")
    obj_dict = {}
    for obj in response['Contents']:
        if obj['Key'].endswith('.csv'):
            print(obj['Key'], end=' ')
            print(obj['LastModified'])
            obj_dict[obj['Key']] = obj['LastModified']
        else:
            pass
    sorted_items = dict(sorted(obj_dict.items(), key=lambda x: x[1], reverse=False))
    print(sorted_items)
    # 키값 중 첫번째 있는 것을 삭제
    s3.delete_object(Bucket=bucket_name, Key=list(sorted_items.keys())[0])


Objects in prefix: at_user_testing_paper/
at_user_testing_paper/AT_USER_TESTING_PAPER_202508291503.csv 2025-08-29 08:09:52+00:00
at_user_testing_paper/AT_USER_TESTING_PAPER_202509301206.csv 2025-09-30 06:04:33+00:00
{'at_user_testing_paper/AT_USER_TESTING_PAPER_202508291503.csv': datetime.datetime(2025, 8, 29, 8, 9, 52, tzinfo=tzutc()), 'at_user_testing_paper/AT_USER_TESTING_PAPER_202509301206.csv': datetime.datetime(2025, 9, 30, 6, 4, 33, tzinfo=tzutc())}

Objects in prefix: at_user_testing_paper_pn/
at_user_testing_paper_pn/AT_USER_TESTING_PAPER_PN_202508291504.csv 2025-08-29 08:10:45+00:00
at_user_testing_paper_pn/AT_USER_TESTING_PAPER_PN_202509300911.csv 2025-09-30 06:04:34+00:00
{'at_user_testing_paper_pn/AT_USER_TESTING_PAPER_PN_202508291504.csv': datetime.datetime(2025, 8, 29, 8, 10, 45, tzinfo=tzutc()), 'at_user_testing_paper_pn/AT_USER_TESTING_PAPER_PN_202509300911.csv': datetime.datetime(2025, 9, 30, 6, 4, 34, tzinfo=tzutc())}

Objects in prefix: at_user_testing_paper_sct/
a

# 새로운 데이터로 교체한 프리픽스 대상 크롤러 동작

## 크롤러 목록 조회

In [112]:
# Glue의 list_crawlers() API는 최대 25개 크롤러 이름만 반환
# NextToken이 있으면 다음 페이지가 있다는 의미
# NextToken이 없을 때까지 반복 호출하여 모든 크롤러 이름을 가져옴
import boto3

glue = boto3.client("glue", region_name="ap-northeast-2")

crawler_names = []
response = glue.list_crawlers()

crawler_names.extend(response.get("CrawlerNames", []))

# NextToken 이 있으면 루프 돌면서 이어서 가져오기
while "NextToken" in response:
    response = glue.list_crawlers(NextToken=response["NextToken"])
    crawler_names.extend(response.get("CrawlerNames", []))

print(crawler_names)
print("총 개수:", len(crawler_names))

['crawler_aggression_norms', 'crawler_aggression_point', 'crawler_at_basic_inquiry_val', 'crawler_at_formula_code', 'crawler_at_low_scale', 'crawler_at_question', 'crawler_at_question_choice', 'crawler_at_question_extra', 'crawler_at_scale_code', 'crawler_at_sub_psy_item', 'crawler_at_user_formula_score', 'crawler_at_user_scale_score', 'crawler_at_user_testing_paper', 'crawler_at_user_testing_paper_pn', 'crawler_at_user_testing_paper_sct', 'crawler_at_user_testing_paper_tr', 'crawler_member', 'crawler_prosociality_norms', 'crawler_prosociality_point', 'crawler_psy_class', 'crawler_psy_estimate', 'crawler_psy_ord', 'crawler_psy_ord_class', 'crawler_psy_target', 'crawler_school_class', 'crawler_school_info', 'crawler_school_student', 'crawler_school_teacher', 'saas_proto_item_forscale', 'saas_proto_scale_tree', 'saas_proto_test_info']
총 개수: 31


In [115]:
# 수많은 크롤러 중 cleaned_target_prefix에 해당하는 크롤러 이름만 추출
target_crawler_names = []
for prefix in cleaned_target_prefix:
    prefix_name = prefix.rstrip('/')
    for crawler_name in crawler_names:
        if prefix_name == re.sub('crawler_', '', crawler_name.lower()):
            target_crawler_names.append(crawler_name)
        else:
            pass
print(target_crawler_names)
print("총 개수:", len(target_crawler_names))

['crawler_at_user_testing_paper', 'crawler_at_user_testing_paper_pn', 'crawler_at_user_testing_paper_sct', 'crawler_at_user_testing_paper_tr', 'crawler_member', 'crawler_psy_class', 'crawler_psy_estimate', 'crawler_psy_ord', 'crawler_psy_ord_class', 'crawler_psy_target', 'crawler_school_class', 'crawler_school_info', 'crawler_school_student', 'crawler_school_teacher']
총 개수: 14


## 스키마 변경 시 동적 속성 변경

In [116]:
# target_crawler_names의 데이터 업데이트 및 삭제에 대한 속성 변경
import time
for crawler in target_crawler_names:
    if crawler.startswith("crawler_"):
        try:
            glue.update_crawler(
                Name=crawler,
                SchemaChangePolicy={
                    "UpdateBehavior": "UPDATE_IN_DATABASE",
                    "DeleteBehavior": "DELETE_FROM_DATABASE"
                }
            )
            print(f"✅ {crawler} 의 SchemaChangePolicy 업데이트 완료")
            time.sleep(1)
        except Exception as e:
            print(f"❌ {crawler} 업데이트 실패: {repr(e)}")

✅ crawler_at_user_testing_paper 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_pn 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_sct 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_tr 의 SchemaChangePolicy 업데이트 완료
✅ crawler_member 의 SchemaChangePolicy 업데이트 완료
✅ crawler_psy_class 의 SchemaChangePolicy 업데이트 완료
✅ crawler_psy_estimate 의 SchemaChangePolicy 업데이트 완료
✅ crawler_psy_ord 의 SchemaChangePolicy 업데이트 완료
✅ crawler_psy_ord_class 의 SchemaChangePolicy 업데이트 완료
✅ crawler_psy_target 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_class 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_info 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_student 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_teacher 의 SchemaChangePolicy 업데이트 완료


In [117]:
# target_crawler_names 크롤러 동작
for crawler in target_crawler_names:
    try:
        glue.start_crawler(Name=crawler)
        print(f"✅ {crawler} 크롤러 시작")
        time.sleep(1)
    except Exception as e:
        print(f"❌ {crawler} 크롤러 시작 실패: {repr(e)}")

✅ crawler_at_user_testing_paper 크롤러 시작
✅ crawler_at_user_testing_paper_pn 크롤러 시작
✅ crawler_at_user_testing_paper_sct 크롤러 시작
✅ crawler_at_user_testing_paper_tr 크롤러 시작
✅ crawler_member 크롤러 시작
✅ crawler_psy_class 크롤러 시작
✅ crawler_psy_estimate 크롤러 시작
✅ crawler_psy_ord 크롤러 시작
✅ crawler_psy_ord_class 크롤러 시작
✅ crawler_psy_target 크롤러 시작
✅ crawler_school_class 크롤러 시작
✅ crawler_school_info 크롤러 시작
✅ crawler_school_student 크롤러 시작
✅ crawler_school_teacher 크롤러 시작
